In [1]:
# hit API 
import requests

# parsing data
import pandas as pd
from datetime import date

# scraping progress
from tqdm import tqdm

# connecting to db
import pymongo
from pymongo import MongoClient
from pymongo.server_api import ServerApi

In [2]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("DANMAR LINES")

In [3]:
# key to map key from json request to key used in db
Mapping = { "Estimated Vessel Departure"  : "ETD",
            "Actual Vessel Departure" : "ATD",
            "Estimated Vessel Arrival" : "ETA",
            "Actual Vessel Arrival" : "ATA"}

In [5]:
list_of_dict = []

for i, bls in enumerate(tqdm(bl_list)):
    
    print("BL in Process: " ,bls)
    
    # hitting API
    url = "https://api.dhl.com/dgff/transportation/shipment-tracking"
    querystring = {"housebill": bls}
    headers = {
        "content-type": "application/json",
        "DHL-API-Key": "qEHPTpIAc13hHfjB6H6gJQ0vEfmP6GhA"
    }

    # pasting json response
    response = requests.request("GET", url, headers=headers, params=querystring)
    json_data = response.json() if response and response.status_code == 200 else None

    temp_dict={}
    
    # list of containers in 1 bl
    list_of_containers=[]
    for i in json_data['ShipmentTracking']['Shipment']['TransportUnits']['TransportUnit']:
        list_of_containers.append(i['TransportUnitID'])

    print(list_of_containers)

    # parsing every data per container
    for c, container in enumerate(tqdm(list_of_containers)):
        current_dict={}
        current_dict.update({"Liners" : "Danmar Lines / DHL"})
        current_dict.update({"BL Number" : bls})
        current_dict.update({"Container Number" : container})
        current_dict.update({"From" : json_data['ShipmentTracking']['Shipment']['Origin']['LocationName'][:json_data['ShipmentTracking']['Shipment']['Origin']['LocationName'].index(",")]})
        current_dict.update({"To" : json_data['ShipmentTracking']['Shipment']['Destination']['LocationName']})
        
        for j, status_dict in enumerate (json_data['ShipmentTracking']['Shipment']['Timestamps']['Timestamp']):
            status_name = status_dict.get('TimestampDescription')
            status_date = status_dict.get('TimestampDateTime')
            case_data = {status_name:status_date[:10]}
            temp_dict.update(case_data)
            
        for old_key, new_key in Mapping.items():
            try:
                current_dict[new_key] = temp_dict[old_key]
            except Exception as e:
                current_dict[new_key] = None
                print("THIS IS ERROR====>",e)

        list_of_dict.append(current_dict)
        print(bls, container,"done")

  0%|          | 0/17 [00:00<?, ?it/s]

BL in Process:  BDOA02869
['CMAU8948938', 'BEAU6259980']


  6%|▌         | 1/17 [00:00<00:08,  1.85it/s]

BDOA02869 CMAU8948938 done
BDOA02869 BEAU6259980 done
BL in Process:  BDOA02949
['FFAU3985639', 'KMTU9355951', 'TEMU8990179']


 12%|█▏        | 2/17 [00:00<00:05,  2.77it/s]

...

THIS IS ERROR====> 'Actual Vessel Arrival'
BDOA02952 GCXU5886255 done
THIS IS ERROR====> 'Actual Vessel Arrival'
BDOA02952 TEMU8979420 done
THIS IS ERROR====> 'Actual Vessel Arrival'
BDOA02952 TXGU5035775 done
THIS IS ERROR====> 'Actual Vessel Arrival'
BDOA02952 FCIU7132020 done
THIS IS ERROR====> 'Actual Vessel Arrival'
BDOA02952 FFAU4200951 done
BL in Process:  BDOA02781
['OOLU4945342', 'CSNU8503995']


 94%|█████████▍| 16/17 [00:03<00:00,  4.44it/s]

BDOA02781 OOLU4945342 done
BDOA02781 CSNU8503995 done
BL in Process:  BDOA02816
['CAAU5812644', 'MSMU8180501']


100%|██████████| 17/17 [00:04<00:00,  4.12it/s]

BDOA02816 CAAU5812644 done
BDOA02816 MSMU8180501 done


In [9]:
list_of_dict[-5]

{'Liners': 'Danmar Lines / DHL',
 'BL Number': 'BDOA02952',
 'Container Number': 'FFAU4200951',
 'From': 'Jakarta',
 'To': 'Cat Lai',
 'ETD': '2023-09-15',
 'ATD': '2023-09-14',
 'ETA': '2023-09-20',
 'ATA': None}

In [10]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(list_of_dict)

Today's Collection Name ===>  all_tracking_Sep-20-2023
Today's Collection Has Been Made
Inserting Many Complete!!


In [23]:
# exporting scraped data to excel
df = pd.DataFrame(list_of_dict)
excel_file_path = 'export excel/DHL.xlsx'
df.to_excel(excel_file_path, index=False)